In [ ]:
import re
from urllib.request import urlopen

In [ ]:
pages = []
for i in range(100):
    start = i*50+1
    pages.append(
        'https://www.imdb.com/search/title?title_type=movie&genres=comedy&start=%d&explore=title_type,genres&ref_=adv_prv'% start
    )

In [ ]:
movie_links = []
for page in pages:
    html = urlopen(page)
    source = html.read()
    links = re.findall(
        '\"(/title/tt[0-9]*)/[\?ref_=adv_li_a-z]*\"'
        ,str(source)[2:-1])
    for link in set(links):
        movie_links.append('https://www.imdb.com' + link)

In [ ]:
with open('./html/imdb1.txt','w') as f:
    for x in movie_links:
        f.write(x+'\n')

In [ ]:
with open('./html/imdb1.txt','r') as f:
    movie_links = f.readlines()

In [ ]:
movie_links = [x.strip('\n') for x in movie_links]

In [1]:
import pandas as pd
from numpy import nan as Nan

In [ ]:
imdb = pd.DataFrame({'name':[],'link':[],'year':[],'subtext':[],'duration':[],'genres':[],'release date':[],
                    'director':[],'writer':[],'stars':[],'ratingCount':[],'avgRating':[]})

for movie in movie_links:
    html = urlopen(movie)
    sourc = html.read()
    source = str(sourc)[2:-1]
    name = re.findall('"title_wrapper">\\\\n<h1 class="[a-z]*">([^<]*)&nbsp;',source)[0]
    link = movie
    year = re.findall('/year/([0-9]*)/',source)
    
    if year:
        year = year[0]
    else:
        year = Nan
    subtext = re.findall('"subtext">\\\\n[ ]*([^ ]*)\\\\n',source)
    if subtext:
        subtext = subtext[0]
    else:
        subtext = Nan
    duration = re.findall('\\\\n[ ]*([0-9]*h* [0-9]*min)\\\\n',source)
    if duration:
        duration = duration[0]
    else:
        duration = re.findall('\\\\n[ ]*([0-9]*h [0-9]*min*)\\\\n',source)
        if duration:
            duration = duration[0]
        else:
            duration = Nan
    genres = ' & '.join(re.findall('genres=([a-z]*)&',source))
    release_date = re.findall('release dates" >([a-zA-Z \(\)0-9]*)\\\\n',source)
    if release_date:
        release_date=release_date[0]
    else:
        release_date = Nan
    rating = re.findall('"aggregateRating": \{(.*)\},\\\\n *"review"',str(sourc)[2:-1])
    if rating:
        rating = rating[0]
        ratingCount = re.findall('"ratingCount": ([0-9]*)',rating)[0]
        avgRating = re.findall('"ratingValue": "(.*)"',rating)[0]
    else:
        ratingCount = Nan
        avgRating = Nan
    fullcredits = 'http://www.imdb.com/title/%s/fullcredits' % re.findall('title/(.*)',movie)[0]
    html1 = urlopen(fullcredits)
    source1 = str(html1.read())[2:-1]
    target = re.findall('fullcredits_content(.*)Produced by',source1)
    if target:
        target = target[0]
        director = re.findall(
            'Directed by(.*)Writing Credits',
            target
        )
        writer = re.findall(
            'Writing Credits(.*)<h4 name="cast"',
            target
        )
        if writer:
            writer = writer[0]
            writer_name = re.findall('/[a-z0-9]+/[?a-z_=0-9]*"\\\\n> ([^</]*)\\\\n',writer)
            job = re.findall('\(([^\(\)]*)\)',writer)
            if len(writer_name) != len(job):
                writer_job = ' & '.join(writer_name)
            else:
                writer_job = ' & '.join([writer_name[i]+' ('+job[i]+') ' for i in range(len(writer_name))])
            if director:
                director = director[0]
                director = ' & '.join(re.findall('/[a-z0-9]+/[?a-z_=0-9]*"\\\\n> ([^</]*)\\\\n',director))
            else:
                director = Nan
        else:
            writer_job = Nan
            director = re.findall('Directed by(.*)<h4 name="cast"',target)
            if director:
                director = director[0]
            else:
                director = re.findall('Directed by(.*)Writing Credits',target)
                if director:
                    director = director[0]
                else:
                    director = Nan
                    writer_name = re.findall('/nm[0-9]*/[?_=a-z0-9]*"\\\\n> ([^<]*)\\\\n',target)
                    job = re.findall('\(([^\(\)]*)\)',target)
                    if len(writer_name) != len(job):
                        writer_job = ' & '.join(writer_name)
                    else:
                        writer_job = ' & '.join([writer_name[i]+' ('+job[i]+') ' for i in range(len(writer_name))])
            if director is not Nan:
                
                director = ' & '.join(re.findall('/[a-z0-9]+/[a-z0-9_=?]*"\\\\n> ([^<]*)\\\\n',director))
            else:
                director = Nan
        stars = re.findall(
            '<h4 name="cast"(.*)',
            target
        )
        if stars:
            stars = stars[0]
            star_names = ' & '.join(re.findall('/[a-z0-9]+/[a-z0-9_=?]*"\\\\n> ([^</]*)\\\\n',stars))
        else:
            stars = Nan
            
    else:
        director = Nan
        writer_job = Nan
        stars = Nan
    tmp = pd.DataFrame({'name':name,'link':link,'year':year,'subtext':subtext,'duration':duration,'genres':genres,
                    'release date':release_date,'director':director,'writer':writer_job,'stars':star_names,
                    'ratingCount':ratingCount,'avgRating':avgRating},index=[i])
    imdb = imdb.append(tmp)

In [6]:
imdb = imdb[['name','year','subtext','duration','genres','director','stars']]

In [24]:
for i in range(imdb.shape[0]):
    if imdb.

name
year
subtext
duration
genres
director
stars


In [21]:
imdb[imdb.year.notna()]['year'] = imdb[imdb.year.notna()].year.apply(lambda x: str(int(x)))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
imdb

,name,year,subtext,duration,genres,director,stars
0,Shazam!,2019.0,PG-13,2h 12min,action & adventure & comedy & action & adventu...,David F. Sandberg,Zachary Levi & Mark Strong & Asher Angel & Jac...
1,The Dirt,2019.0,TV-MA,1h 47min,biography & comedy & drama & biography & comed...,Jeff Tremaine,Machine Gun Kelly & Erin Ownbey & Douglas Boot...
2,The Dead Don\'t Die,2019.0,NaN,NaN,comedy & horror & comedy & horror,Jim Jarmusch,Chlo\xc3\xab Sevigny & Adam Driver & Selena Go...
3,Once Upon a Time in Hollywood,2019.0,NaN,NaN,comedy & drama & comedy & drama,Quentin Tarantino,Timothy Olyphant & Margot Robbie & Leonardo Di...
4,The Beach Bum,2019.0,R,1h 35min,comedy & comedy,Harmony Korine,Matthew McConaughey & Snoop Dogg & Isla Fisher...
5,How to Train Your Dragon: The Hidden World,2019.0,PG,1h 44min,animation & action & adventure & animation & a...,Dean DeBlois,Jay Baruchel & America Ferrera & F. Murray Abr...
6,Unicorn Store,2017.0,TV-PG,1h 32min,comedy & drama & comedy & drama,Brie Larson,Brie Larson & Samuel L. Jackson & Joan Cusack ...
7,Green Book,2018.0,PG-13,2h 10min,biography & comedy & drama & biography & comed...,Peter Farrelly,Viggo Mortensen & Mahershala Ali & Linda Carde...
8,Mary Poppins Returns,2018.0,PG,2h 10min,comedy & family & fantasy & comedy & family & ...,Rob Marshall,Emily Blunt & Lin-Manuel Miranda & Ben Whishaw...
9,Mamma Mia! Here We Go Again,2018.0,PG-13,1h 54min,comedy & musical & romance & comedy & musical ...,Ol Parker,Amanda Seyfried & Andy Garcia & Celia Imrie & ...


In [8]:
imdb.to_csv('imdb.csv',index=False)